In [1]:
import pandas as pd
import torch

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
y = train['label']

x = train.drop(columns='label')

In [4]:
train['label'].unique().sum()

np.int64(45)

In [5]:
y = torch.tensor(y)

train_x = torch.tensor(x.values, dtype=torch.float32)

test_x = torch.tensor(test.values, dtype=torch.float32)

In [12]:
train_x.shape

torch.Size([42000, 784])

In [7]:
model = torch.nn.Sequential(
                            torch.nn.Linear(in_features= 28 * 28, out_features=392),
                            torch.nn.ReLU(),
                            torch.nn.Linear(392, 194),
                            torch.nn.ReLU(),
                            torch.nn.Linear(194, 97),
                            torch.nn.ReLU(),
                            torch.nn.Linear(97, 48),
                            torch.nn.ReLU(),
                            torch.nn.Linear(48, 45))

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = torch.nn.CrossEntropyLoss()

In [9]:
for t in range(250):
    y_pred = model(train_x)
    
    loss = loss_fn(y_pred, y)
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    # Calcula as previsões (classe com maior logit)
    predicted = torch.argmax(y_pred, dim=1)
    correct = (predicted == y).sum().item()
    accuracy = correct / train_x.size(0) * 100
    
    print(f'Epoch {t}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.2f}%')


Epoch 0, Loss: 5.2336, Accuracy: 0.11%
Epoch 1, Loss: 3.6779, Accuracy: 10.49%
Epoch 2, Loss: 3.0428, Accuracy: 18.65%
Epoch 3, Loss: 2.4039, Accuracy: 39.58%
Epoch 4, Loss: 1.8782, Accuracy: 52.17%
Epoch 5, Loss: 1.4929, Accuracy: 60.99%
Epoch 6, Loss: 1.1689, Accuracy: 65.55%
Epoch 7, Loss: 0.9165, Accuracy: 74.78%
Epoch 8, Loss: 0.7862, Accuracy: 75.31%
Epoch 9, Loss: 0.6809, Accuracy: 79.85%
Epoch 10, Loss: 0.6299, Accuracy: 82.20%
Epoch 11, Loss: 0.6341, Accuracy: 81.27%
Epoch 12, Loss: 0.5727, Accuracy: 82.80%
Epoch 13, Loss: 0.5023, Accuracy: 85.30%
Epoch 14, Loss: 0.4840, Accuracy: 86.32%
Epoch 15, Loss: 0.4489, Accuracy: 87.46%
Epoch 16, Loss: 0.4366, Accuracy: 87.75%
Epoch 17, Loss: 0.3988, Accuracy: 88.84%
Epoch 18, Loss: 0.3882, Accuracy: 88.94%
Epoch 19, Loss: 0.3581, Accuracy: 90.09%
Epoch 20, Loss: 0.3409, Accuracy: 90.38%
Epoch 21, Loss: 0.3294, Accuracy: 90.64%
Epoch 22, Loss: 0.3151, Accuracy: 91.06%
Epoch 23, Loss: 0.2894, Accuracy: 91.86%
Epoch 24, Loss: 0.2866, Acc

KeyboardInterrupt: 

In [10]:
y_test= model(test_x)

predicted = torch.argmax(y_test, dim=1)

result = pd.DataFrame()
result['ImageId'] = range(1, len(predicted) + 1)
result['Label'] = predicted

In [11]:
result.to_csv('MNIST_result.csv', index=False)